In [ ]:
import netCDF4 as nc
from netCDF4 import Dataset
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde, poisson
import tropycal.tracks as tracks
import pandas as pd
import numpy as np
import datetime as dt
import os
import re
import geopandas as gpd
import geodatasets
from shapely import geometry, wkb, wkt
import multiprocessing as mp
import sys
import threading
from queue import Queue
import xarray as xr
import pathlib
import sqlite3
import pickle
from scipy.stats import mannwhitneyu

Western Pacific (WP) locations: 5–60°N 100°–180°E

Current

In [ ]:
s = re.compile('^MRI_HPB_m+\d{3}.nc$')
file_dirs = os.listdir('data')
file_dirs
# data_files = [f for f in file_dirs if s.match(f)]
data_files_dirs = [pathlib.Path('data') / pathlib.Path(f) for f in file_dirs if s.match(f)]
# data_files
len(data_files_dirs)

In [ ]:
test_storms = xr.open_dataset(data_files_dirs[1])
i = 0
test_storms.track_lat[i]
# for lat in test_storms.track_lat[i]:
#     print(lat.values)
test_storms.where((test_storms.track_lat >= 5) & (test_storms.track_lat <= 60) & (test_storms.track_lon >= 100) & (test_storms.track_lon <= 180), drop=True)
# np.isnat(test_storms.track_time)
# t = geometry.LineString(zip(test_storms.track_lon[i], test_storms.track_lat[i]))
# b = t.wkb
# b.hex()
# tb = wkb.loads(b, hex=True)
# tb
# np.datetime_as_string(test_storms.track_time[1][3].values, unit='m')
# data_files_dirs[1].stem
# type(test_storms.track_time[0].values[0])
test_storms.track_time[0].values[0] >= np.datetime64('2000-01-01')

In [ ]:

# storm_geo = mp.Manager().list()
def create_storm_dict_task(storm_i, storms_d, storms, stem):
    time_l = storms.track_time[storm_i].where(~np.isnat(storms.track_time[storm_i]), drop=True).values
    lon_l = storms.track_lon[storm_i].where(~np.isnan(storms.track_lon[storm_i]), drop=True).values
    lat_l = storms.track_lat[storm_i].where(~np.isnan(storms.track_lat[storm_i]), drop=True).values
    pres_l = storms.track_pres[storm_i].where(~np.isnan(storms.track_pres[storm_i]), drop=True).values
    wind_l = storms.track_wind[storm_i].where(~np.isnan(storms.track_wind[storm_i]), drop=True).values
    # for i in range(len(storms.track_time[storm_i])):
    #     # lat = storms.track_lat[storm_i][i]
    #     # lon = storms.track_lon[storm_i][i]
    #     if not np.isnan(storms.track_lon[storm_i][i]):
    #         coords.append((storms.track_lon[storm_i][i], storms.track_lat[storm_i][i]))
    #     else:
    #         break
    if len(lat_l) <= 1:
        # print(f'{stem}_{storm_i} has no valid coordinates')
        return
    storms_d[f'{stem}_{storm_i}'] = {
        # 'time': storms[i]['time'],
        # 'geometry': geometry.LineString((lat, lon) for lat, lon in zip(storms.track_lon[storm_i], storms.track_lat[storm_i]) if (not np.isnan(lat) and not np.isnan(lon))),
        # 'geometry': geometry.LineString(coords),
        # 'featurecla': 'storm',
        # 'scalerank': 1,
        # 'min_zoom': 1,
        'time': time_l,
        'lat': lat_l,
        'lon': lon_l,
        'pres': pres_l,
        'wind': wind_l
        
    }
    # geometry_d.append(geometry.LineString((lat, lon) for lat, lon in zip(storms.track_lon[storm_i], storms.track_lat[storm_i]) if (not np.isnan(lat) and not np.isnan(lon))))
    # print(storm_i)
def generate_storm_dict(stem, storms_d):
    storms = xr.open_dataset(f'data/{stem}.nc')
    filtered_storms = storms.where((storms.track_lat >= 5) & (storms.track_lat <= 60) & (storms.track_lon >= 100) & (storms.track_lon <= 180) & (storms.track_time >= np.datetime64('2000-01-01')), drop=True)
    # filtered_storms = storms.where((storms.track_lat >= 5) & (storms.track_lat <= 60) & (storms.track_lon >= 60) & (storms.track_lon <= 280), drop=True)
    # print(filtered_storms)
    print(f"{stem}\n", end='')
    for storm_i in range(len(filtered_storms.storm)):
        create_storm_dict_task(storm_i, storms_d, filtered_storms, stem)
    # pool = mp.Pool(mp.cpu_count())
    # pool.starmap(create_storm_dict_task, ((storm_i, storms_d, filtered_storms, stem) for storm_i in range(len(filtered_storms.storm))))
    # pool.close()
    # pool.join()
# processes = []
# for data in data_files_dirs:
#     storms = xr.open_dataset(data)
#     generate_storm_dict(storms, data.stem)


    # processes.append(mp.Process(target=generate_geometry, args=(storms, data.stem)))
    # generate_geometry(storms, data.stem)
# for p in processes:
#     p.start()
# for p in processes:
#     p.join()
# storm_geo = storm_geo._getvalue()
# print(storm_geo)
# generate_geometry(test_storms, "test")
# create_geometry(1203, storm_geo, test_storms, "test")

In [ ]:
storms_d = mp.Manager().dict()
pool = mp.Pool(mp.cpu_count())
pool.starmap(generate_storm_dict, ([data.stem, storms_d] for data in data_files_dirs))
pool.close()
pool.join()
storms_d = storms_d._getvalue()

In [ ]:
with open('storms_d.p', 'wb') as f:
    pickle.dump(storms_d, f)


In [ ]:
with open('storms_d.p', 'rb') as f:
    storms_d = pickle.load(f)
storms_d

In [ ]:
# world = gpd.read_file(geodatasets.get_path('naturalearth.land'))
# df = pd.DataFrame.from_dict(storms_d, orient='index')
# roi = gpd.GeoDataFrame(df, geometry='geometry', crs=world.crs)
# roi
# world_roi = pd.concat([world, roi])
# world_roi

In [ ]:
def createGrid(storms, n, res):
    storms_df = pd.DataFrame.from_dict(storms, orient='index')
    lat_max = storms_df['lat'].apply(np.max).max()
    lat_min = storms_df['lat'].apply(np.min).min()
    lon_max = storms_df['lon'].apply(np.max).max()
    lon_min = storms_df['lon'].apply(np.min).min()
    time_min = storms_df['time'].apply(np.min).min()
    time_max = storms_df['time'].apply(np.max).max()
    year_num = (time_max-time_min).days/365.2425
    grids = np.zeros((round((lon_max-lon_min)/res)+1, round((lat_max-lat_min)/res)+1))
    for i in storms:
        # cell_coords = {}
        coords = []
        for lat, lon in zip(storms[i]['lat'], storms[i]['lon']):
                # grids[round((lon-lon_min)/res), round((lat-lat_min)/res)] += 1
                if (round((lon-lon_min)/res), round((lat-lat_min)/res)) not in coords:
                    coords.append((round((lon-lon_min)/res), round((lat-lat_min)/res)))
        for i, j in coords:
            grids[i, j] += 1
                
    grids = grids/year_num * 10/n
    # grid_cells = []
    X = []
    Y = []
    C = np.zeros((grids.shape[1], grids.shape[0]))

    for i in range(grids.shape[0]):
        X.append(lon_min + (i+0.5)*res)
    for i in range(grids.shape[1]):
        Y.append(lat_min + (i+0.5)*res)
    for i in range(grids.shape[0]):
        for j in range(grids.shape[1]):
            C[j, i] = grids[i, j]
            
    lon_mesh, lat_mesh = np.meshgrid(X, Y)
    return lon_mesh, lat_mesh, C, lat_min, lat_max, lon_min, lon_max

def plot_map(lon_mesh, lat_mesh, C, lat_min, lat_max, lon_min, lon_max):
    world = gpd.read_file(geodatasets.get_path('naturalearth.land'))
    norm = plt.matplotlib.colors.Normalize(vmin=C.min(), vmax=C.max())
    fig, ax = plt.subplots()
    world.plot(color='gray', edgecolor='black', ax=ax)
    ax.figure.set_size_inches(20, 15)
    ax.set_xlim(100, 180)
    ax.set_ylim(5, 60)
    cmap = plt.get_cmap('rainbow', 256)
    cmap.set_bad((0, 0, 0, 0))
    colors = cmap(np.linspace(0, 1, 256))
    # colors[0][3] = 0
    cmap = plt.matplotlib.colors.ListedColormap(colors)
    pcm = ax.pcolormesh(lon_mesh, lat_mesh, C, cmap=cmap, shading='auto', alpha=0.8, norm=norm)
    num_ticks = 6
    cbar = fig.colorbar(pcm, ax=ax, orientation='vertical',
                        extend='max',
                        ticks=[round(i, 2) for i in np.linspace(norm.vmin, norm.vmax, num_ticks)],
                        pad=0.02, 
                        fraction=0.03)
    cbar.ax.set_title('[No./10yr]')
    cbar.ax.set_yticklabels([f'{round(i, 2)}' for i in np.linspace(norm.vmin, norm.vmax, num_ticks)])
    plt.tight_layout()

def plot(storms, n, res):
    # res = 2.25
    # res = 1.
    world = gpd.read_file(geodatasets.get_path('naturalearth.land'))
    # storms = storms_d
    
    lon_mesh, lat_mesh, C, lat_min, lat_max, lon_min, lon_max = createGrid(storms, n, res)
    
    norm = plt.matplotlib.colors.LogNorm(vmin=0.1, vmax=C.max())
    fig, ax = plt.subplots()
    world.plot(color='gray', edgecolor='black', ax=ax)
    ax.figure.set_size_inches(20, 15)
    ax.set_xlim(100, 180)
    ax.set_ylim(5, 60)
    cmap = plt.get_cmap('rainbow', 256)
    colors = cmap(np.linspace(0, 1, 256))
    colors[0][3] = 0
    cmap = plt.matplotlib.colors.ListedColormap(colors)
    pcm = ax.pcolormesh(lon_mesh, lat_mesh, C, cmap=cmap, shading='auto', alpha=0.8, norm=norm)
    num_ticks = 6
    cbar = fig.colorbar(pcm, ax=ax, orientation='vertical',
                        extend='max',
                        ticks=[round(i, 2) for i in np.linspace(norm.vmin, norm.vmax, num_ticks)],
                        pad=0.02, 
                        fraction=0.03)
    cbar.ax.set_title('[No./10yr]')
    cbar.ax.set_yticklabels([f'{round(i, 2)}' for i in np.linspace(norm.vmin, norm.vmax, num_ticks)])
    plt.tight_layout()


In [ ]:
plot(storms_d, 100, 2.25)

In [ ]:
s = re.compile('^MRI_HFB')
file_dirs = os.listdir('data')
file_dirs
# data_files = [f for f in file_dirs if s.match(f)]
data_files_dirs = [pathlib.Path('data') / pathlib.Path(f) for f in file_dirs if s.match(f)]
# data_files
len(data_files_dirs)

In [ ]:
storms_4K_d = mp.Manager().dict()
pool = mp.Pool(mp.cpu_count())
pool.starmap(generate_storm_dict, ([data.stem, storms_4K_d] for data in data_files_dirs))
pool.close()
pool.join()
storms_4K_d = storms_4K_d._getvalue()

In [ ]:
with open('storms_4K_d.p', 'wb') as f:
    pickle.dump(storms_4K_d, f)

In [ ]:
with open('storms_4K_d.p', 'rb') as f:
    storms_4K_d = pickle.load(f)
storms_4K_d

In [ ]:
plot(storms_4K_d, 90, 2.25)

In [ ]:
obs = xr.open_dataset('data/IBTrACS.WP.v04r01.nc')
obs = obs.where((obs.lat >= 5) & (obs.lat <= 60) & (obs.lon >= 100) &  (obs.lon <= 180), drop=True)

In [ ]:
storms_obs_d = {}
for i in range(len(obs.storm)):
    time = obs.time[i].where(~np.isnan(obs.usa_lat[i]), drop=True).values
    # lat = obs.usa_lat[i].where(~np.isnan(obs.usa_lat[i]), drop=True).values
    # lon = obs.usa_lon[i].where(~np.isnan(obs.usa_lon[i]), drop=True).values
    lat = obs.lat[i].where(~np.isnan(obs.lat[i]), drop=True).values
    lon = obs.lon[i].where(~np.isnan(obs.lon[i]), drop=True).values
    if len(time) != 0:
        storms_obs_d[f'IBTrACS_{i}'] = {
            'time': time,
            'lat': lat,
            'lon': lon,
        }

In [ ]:
with open('storms_obs_d.p', 'wb') as f:
    pickle.dump(storms_obs_d, f)

In [ ]:
with open('storms_obs_d.p', 'rb') as f:
    pickle.load(f)
storms_obs_d

In [ ]:
plot(storms_obs_d, 1, 2.25)

In [ ]:
s = re.compile('^MRI_HPB_m+\d{3}.nc$')
file_dirs = os.listdir('data')
file_dirs
# data_files = [f for f in file_dirs if s.match(f)]
data_files_dirs = [pathlib.Path('data') / pathlib.Path(f) for f in file_dirs if s.match(f)]
# data_files
len(data_files_dirs)

In [ ]:
current_d_keys = storms_d.keys()
current_storms_df = pd.DataFrame.from_dict(storms_d, orient='index')
lat_max = current_storms_df['lat'].apply(np.max).max()
lat_min = current_storms_df['lat'].apply(np.min).min()
lon_max = current_storms_df['lon'].apply(np.max).max()
lon_min = current_storms_df['lon'].apply(np.min).min()
time_min = current_storms_df['time'].apply(np.min).min()
time_max = current_storms_df['time'].apply(np.max).max()

In [ ]:
grid_current = np.empty((round((lon_max-lon_min)/2.25)+1, round((lat_max-lat_min)/2.25)+1), dtype=list)
grid_current[:] = [[[] for _ in range(grid_current.shape[1])] for _ in range(grid_current.shape[0])]
for current_data_i, current_data in enumerate(data_files_dirs):
    stem = current_data.stem
    re_exp = re.compile(f'^{stem}.+')
    filtered_key = [i for i in current_d_keys if re_exp.match(i)]
    temp_grid = np.zeros((round((lon_max-lon_min)/2.25)+1, round((lat_max-lat_min)/2.25)+1))
    for key in filtered_key:
        coords = []
        for lat, lon in zip(storms_d[key]['lat'], storms_d[key]['lon']):
            if (round((lon-lon_min)/2.25), round((lat-lat_min)/2.25)) not in coords:
                coords.append((round((lon-lon_min)/2.25), round((lat-lat_min)/2.25)))
        for i, j in coords:
            temp_grid[i, j] += 1
    for i in range(temp_grid.shape[0]):
        for j in range(temp_grid.shape[1]):
            if temp_grid[i, j] > 0:
                grid_current[i, j].append(temp_grid[i, j])
grid_current

In [ ]:
s = re.compile('^MRI_HFB')
file_dirs = os.listdir('data')
file_dirs
# data_files = [f for f in file_dirs if s.match(f)]
future_data_files_dirs = [pathlib.Path('data') / pathlib.Path(f) for f in file_dirs if s.match(f)]
# data_files
len(future_data_files_dirs)

In [ ]:
future_d_keys = storms_4K_d.keys()
future_storms_df = pd.DataFrame.from_dict(storms_4K_d, orient='index')
lat_max = future_storms_df['lat'].apply(np.max).max()
lat_min = future_storms_df['lat'].apply(np.min).min()
lon_max = future_storms_df['lon'].apply(np.max).max()
lon_min = future_storms_df['lon'].apply(np.min).min()
time_min = future_storms_df['time'].apply(np.min).min()
time_max = future_storms_df['time'].apply(np.max).max()

In [ ]:
grid_future = np.empty((round((lon_max-lon_min)/2.25)+1, round((lat_max-lat_min)/2.25)+1), dtype=list)
grid_future[:] = [[[] for _ in range(grid_future.shape[1])] for _ in range(grid_future.shape[0])]
for future_data_i, future_data in enumerate(future_data_files_dirs):
    stem = future_data.stem
    re_exp = re.compile(f'^{stem}.+')
    filtered_key = [i for i in future_d_keys if re_exp.match(i)]
    temp_grid = np.zeros((round((lon_max-lon_min)/2.25)+1, round((lat_max-lat_min)/2.25)+1))
    for key in filtered_key:
        coords = []
        for lat, lon in zip(storms_4K_d[key]['lat'], storms_4K_d[key]['lon']):
            if (round((lon-lon_min)/2.25), round((lat-lat_min)/2.25)) not in coords:
                coords.append((round((lon-lon_min)/2.25), round((lat-lat_min)/2.25)))
        for i, j in coords:
            temp_grid[i, j] += 1
    for i in range(temp_grid.shape[0]):
        for j in range(temp_grid.shape[1]):
            if temp_grid[i, j] > 0:
                grid_future[i, j].append(temp_grid[i, j])
grid_future

In [ ]:
grid = np.zeros((grid_current.shape[0], grid_current.shape[1]))
for i in range(grid.shape[0]):
    for j in range(grid.shape[1]):
        try:
            stat, p = mannwhitneyu(grid_future[i, j], grid_current[i, j])
            if p < 0.01:
                grid[i, j] = np.mean(grid_future[i, j]) - np.mean(grid_current[i, j])
        except:
            # print(grid_current[i, j], grid_future[i, j])
            pass
grid

In [ ]:
grid = grid / (time_max-time_min).days * 365.2425 * 10
grid

In [ ]:
X = []
Y = []
for i in range(grid.shape[0]):
    X.append(lon_min + (i+0.5)*2.25)
for i in range(grid.shape[1]):
    Y.append(lat_min + (i+0.5)*2.25)
lon_mesh, lat_mesh = np.meshgrid(X, Y)
grid = np.ma.masked_equal(grid, 0)
plot_map(lon_mesh, lat_mesh, grid.T, lat_min, lat_max, lon_min, lon_max)